220228 과제

# library

In [2]:
#데이터 분석 라이브러리
import numpy as np
print("numpy version: {}". format(np.__version__))

import pandas as pd

#시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(mpl.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#딥러닝 라이브러리
import tensorflow as tf

#기타 라이브러리
import os
import random
import time

#경고 에러 무시
import warnings
warnings.filterwarnings('ignore')
print('-'*50)

#시간 확인을 용이하게 해주는 라이브러리
from tqdm import tqdm

#결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from varname import nameof

numpy version: 1.19.5
matplotlib version: 3.4.2
seaborn version: 0.11.1
--------------------------------------------------


# GPU

In [2]:
#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Method2
x는 d-10부터 쌓여있음을 잊으면 안됨.<br>
method2는 음수값으로 정렬해서 위에서부터 top10<br>

In [8]:
x = np.load('/project/LSH/x_(7727,10,4068).npy')
x[:,:5,0].shape # d10부터 d6까지 5개 날짜 # 원시간
x[:,5:,0].shape # d5부터 d1까지 5개 날짜 # 근시간

# 원시간: :5를 바꿔서 예측에 사용하는 것.
# 근시간: 5:를 바꿔서 예측에 사용하는 것.

(7727, 5)

In [ ]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')


# 2) MODEL
# with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
#     tf.config.experimental.set_virtual_device_configuration(gpus[1], 
#                                                             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

from keras.models import load_model
lstm2 = load_model('/project/LSH/model/allfit_ep500/allfit_ep500_seed42-17-0.7619.hdf5')


# 3) preds
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_entropy_dic = {} # 원시간_method1_entropy
preds2_dic = {} # 원시간_method1_부록

preds3_entropy_dic = {} # 근시간_method1_entropy
preds3_dic = {} # 근시간_method1_부록



for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = x[:,:,k].copy()
    
    # 원시간
    # all_to_1
    x[:,:5,k] = 1

    # COMPUTE pred WITH FEATURE K all_to_1, preds2_mean_list
    preds2_1_원 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_1_원 = np.mean(preds2_1_원)

    # all_to_0
    x[:,:5,k] = 0 #어차피 all change라서 save_col 안하고 해도 됨.

    # COMPUTE pred WITH FEATURE K all_to_0, preds2_mean_list
    preds2_0_원 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_0_원 = np.mean(preds2_0_원)
    
    diff1 = mean_preds2_1_원 - mean_preds2_0_원 # 값 한개
    
    preds2_entropy_dic[features[k]] = diff1 * entropy[k]
    preds2_dic[features[k]] = diff1

    
    x[:,:,k] = save_col
    
    
    
    # 근시간
    # all_to_1
    x[:,5:,k] = 1

    # COMPUTE pred WITH FEATURE K all_to_1, preds2_mean_list
    preds2_1_근 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_1_근 = np.mean(preds2_1_근)

    # all_to_0
    x[:,5:,k] = 0 #어차피 all change라서 save_col 안하고 해도 됨.

    # COMPUTE pred WITH FEATURE K all_to_0, preds2_mean_list
    preds2_0_근 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_0_근 = np.mean(preds2_0_근)
    
    diff2 = mean_preds2_1_근 - mean_preds2_0_근 # 값 한개
    
    preds3_entropy_dic[features[k]] = diff2 * entropy[k]
    preds3_dic[features[k]] = diff2


    x[:,:,k] = save_col

#     break

In [ ]:
dic1 = preds2_entropy_dic.copy()
dic2 = preds2_dic.copy()

dic1
dic2

df1 = pd.DataFrame(dic1.items(), columns=['feature', 'method1_entropy'])
df1

df2 = pd.DataFrame(dic2.items(), columns=['feature', 'method1_부록']) #hp columns는 column name 설정 
df2



'############################################'
dic3 = preds3_entropy_dic.copy()
dic4 = preds3_dic.copy()

dic3
dic4


df3 = pd.DataFrame(dic3.items(), columns=['feature', 'method1_entropy'])
df3

df4 = pd.DataFrame(dic4.items(), columns=['feature', 'method1_부록']) #hp columns는 column name 설정 
df4